# Synthetic Data Generator — Real-World Reconciliation (v2)

This notebook generates realistic, messy financial data for reconciliation.

In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import uuid

## Helper Functions

In [2]:

def random_date(start, end):
    delta = end - start
    return start + timedelta(days=random.randint(0, delta.days))

def random_narration(customer_id, channel):
    templates = {
        "CHEQUE": [
            f"Cheque deposit for member {customer_id}",
            f"CHQ DEP MBR {customer_id}",
            f"Cheque dep member {customer_id}"
        ],
        "MPESA": [
            f"Mobile payment member {customer_id}",
            f"MPESA {customer_id}",
            f"M-PESA PAY {customer_id}"
        ],
        "CASH": [
            f"Cash deposit acc {customer_id}",
            f"CASH DEP {customer_id}"
        ]
    }
    return random.choice(templates[channel])


## Generate Payments System Data

In [3]:

np.random.seed(42)
random.seed(42)

n_payments = 8000
channels = ["CHEQUE", "MPESA", "CASH"]
statuses = ["SUCCESS", "FAILED"]

payments = []

start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 3, 31)

for _ in range(n_payments):
    cust_id = random.randint(20000, 29999)
    channel = random.choice(channels)
    amount = round(random.uniform(500, 50000), 2)
    status = random.choices(statuses, weights=[0.9, 0.1])[0]

    payments.append({
        "payment_ref": f"PS-{uuid.uuid4().hex[:10]}",
        "payment_timestamp": random_date(start_date, end_date),
        "value_date": random_date(start_date, end_date).date(),
        "amount": amount,
        "currency": "KES",
        "payment_channel": channel,
        "customer_id": str(cust_id),
        "narration": random_narration(cust_id, channel),
        "payment_status": status,
        "entered_by": random.choice(["TELLER", "SYSTEM", "BATCH"]),
        "branch_code": f"BR{random.randint(1,20):02d}",
        "expected_fee": round(amount * random.uniform(0.002, 0.01), 2)
    })

payments_df = pd.DataFrame(payments)
payments_df.head()


,payment_ref,payment_timestamp,value_date,amount,currency,payment_channel,customer_id,narration,payment_status,entered_by,branch_code,expected_fee
0,PS-4775870d89,2024-01-18,2024-01-14,37206.75,KES,CHEQUE,21824,Cheque dep member 21824,SUCCESS,BATCH,BR18,100.29
1,PS-d13de5036a,2024-03-05,2024-03-18,1974.96,KES,CHEQUE,26912,Cheque deposit for member 26912,SUCCESS,BATCH,BR07,15.26
2,PS-8955a11f87,2024-01-01,2024-01-21,11411.81,KES,MPESA,28928,M-PESA PAY 28928,SUCCESS,SYSTEM,BR11,48.19
3,PS-3d40654f2f,2024-02-15,2024-02-14,5559.41,KES,MPESA,23527,M-PESA PAY 23527,SUCCESS,SYSTEM,BR02,43.57
4,PS-6163d6dd33,2024-03-11,2024-02-07,48669.23,KES,CHEQUE,28785,Cheque dep member 28785,SUCCESS,BATCH,BR12,322.13


## Generate Bank Statement Data

In [4]:

bank_txns = []

for _, row in payments_df.sample(frac=0.85).iterrows():
    lag_days = random.randint(0, 3)
    fee = round(row["amount"] * random.uniform(0.001, 0.008), 2)

    bank_txns.append({
        "bank_txn_ref": f"BK-{uuid.uuid4().hex[:10]}",
        "posting_date": (row["payment_timestamp"] + timedelta(days=lag_days)).date(),
        "value_date": (row["payment_timestamp"] + timedelta(days=lag_days)).date(),
        "amount": round(row["amount"] - fee, 2),
        "currency": "KES",
        "debit_credit": "CR",
        "narration": row["narration"]
            .replace("member", "MBR")
            .replace("Cheque", "CHQ")
            .replace("deposit", "DEP"),
        "bank_branch": f"BBR{random.randint(1,15):02d}",
        "source_system": row["payment_channel"],
        "fees_deducted": fee
    })

bank_df = pd.DataFrame(bank_txns)
bank_df.head()


,bank_txn_ref,posting_date,value_date,amount,currency,debit_credit,narration,bank_branch,source_system,fees_deducted
0,BK-5e80e11e2a,2024-02-08,2024-02-08,44732.35,KES,CR,CHQ DEP MBR 25164,BBR15,CHEQUE,250.58
1,BK-765196257a,2024-03-04,2024-03-04,5269.51,KES,CR,Mobile payment MBR 25173,BBR08,MPESA,28.68
2,BK-3556cf8643,2024-01-31,2024-01-31,46172.94,KES,CR,Mobile payment MBR 29348,BBR15,MPESA,255.71
3,BK-322fd7af6b,2024-02-02,2024-02-02,34323.18,KES,CR,CHQ DEP MBR 24429,BBR13,CHEQUE,249.92
4,BK-d7b42b4169,2024-03-12,2024-03-12,34351.32,KES,CR,CHQ DEP for MBR 26334,BBR09,CHEQUE,67.19


## Generate Refunds Data

In [5]:

refunds = []

failed_payments = payments_df[payments_df["payment_status"] == "FAILED"].sample(frac=0.7)

for _, row in failed_payments.iterrows():
    refunds.append({
        "refund_ref": f"RF-{uuid.uuid4().hex[:8]}",
        "refund_timestamp": row["payment_timestamp"] + timedelta(days=random.randint(1,5)),
        "refund_amount": row["amount"] * random.choice([1, 1.05]),
        "currency": "KES",
        "refund_channel": row["payment_channel"],
        "customer_id": row["customer_id"],
        "narration": f"Refund {row['narration']}",
        "refund_reason": random.choice(["FAILED_TXN", "DUPLICATE"]),
        "approved_by": random.choice(["SUPERVISOR", "SYSTEM"]),
        "linked_payment_hint": row["narration"],
        "refund_status": "PROCESSED"
    })

refunds_df = pd.DataFrame(refunds)
refunds_df.head()


,refund_ref,refund_timestamp,refund_amount,currency,refund_channel,customer_id,narration,refund_reason,approved_by,linked_payment_hint,refund_status
0,RF-c3b7a4e2,2024-02-07,50891.4525,KES,CASH,25162,Refund CASH DEP 25162,FAILED_TXN,SYSTEM,CASH DEP 25162,PROCESSED
1,RF-e561d983,2024-03-10,26735.4150,KES,CASH,25443,Refund CASH DEP 25443,FAILED_TXN,SUPERVISOR,CASH DEP 25443,PROCESSED
2,RF-a416c3ae,2024-03-02,37968.1800,KES,CHEQUE,26647,Refund Cheque dep member 26647,DUPLICATE,SUPERVISOR,Cheque dep member 26647,PROCESSED
3,RF-2d578885,2024-02-25,22270.7520,KES,CHEQUE,21193,Refund CHQ DEP MBR 21193,DUPLICATE,SYSTEM,CHQ DEP MBR 21193,PROCESSED
4,RF-729ab2d7,2024-02-21,9956.2680,KES,CHEQUE,26801,Refund Cheque deposit for member 26801,FAILED_TXN,SYSTEM,Cheque deposit for member 26801,PROCESSED


## Save Datasets

In [6]:

payments_df.to_csv("C:/Payments Reconciliation/payments_system_v2.csv", index=False)
bank_df.to_csv("C:/Payments Reconciliation/bank_statement_v2.csv", index=False)
refunds_df.to_csv("C:/Payments Reconciliation/refunds_v2.csv", index=False)

print("Files generated successfully")


Files generated successfully
